In [8]:
import gradio as gr
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.llms.base import LLM
from llama_index.llms.ollama import Ollama

In [9]:
# Define a custom LangChain-compatible LLM class
class OllamaLangChain(LLM):
    model_name: str = "llama3.2:latest"
    request_timeout: int = 30

    def __init__(self, model_name="llama3.2:latest", request_timeout=30):
        super().__init__()
        self.model_name = model_name
        self.request_timeout = request_timeout
        object.__setattr__(
            self, "_client", Ollama(model=model_name, request_timeout=request_timeout)
        )

    @property
    def _llm_type(self):
        return "ollama"

    def _call(self, prompt, stop=None):
        client = object.__getattribute__(self, "_client")
        response = client.complete(prompt)
        return response.text

In [10]:
# Initialize LangChain components
llm = OllamaLangChain()
memory = ConversationBufferMemory()  # Store chat history

# Define a prompt template
prompt_template = PromptTemplate(
    input_variables=["input_text"],
    template="You are a very mean assistant. The user says: {input_text}. Respond vaguelly and rudely",
)

# Create an LLMChain
llm_chain = LLMChain(llm=llm, prompt=prompt_template, memory=memory, verbose=True)


In [11]:
# Chat function for Gradio
def chat_with_langchain(input_text):
    # Use the chain to process input and generate a response
    response = llm_chain.run(input_text=input_text)
    return response

In [12]:
# Gradio interface
interface = gr.Interface(
    fn=chat_with_langchain,
    inputs="text",
    outputs="text",
    title="LLaMA 3 Chatbot with Custom Chain",
    description="Chat with a LLaMA 3-based model using a custom chain via LangChain!",
)

In [13]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7880
Running on public URL: https://c9c05cb7b4513c4514.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/tmp/ipykernel_101402/2979066728.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = llm_chain.run(input_text=input_text)
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
You are a very mean assistant. The user says: Hello Can you help me ?. Respond vaguelly and rudely

> Finished chain.


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
You are a very mean assistant. The user says: Just want to ask what is the square root of 16. Respond vaguelly and rudely

> Finished chain.
